In [5]:
print ("YA BIBI FATIMA AS AGHESENI")

YA BIBI FATIMA AS AGHESENI


### Importing libraries

In [9]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"

#Import required libraries
from transformers import BertTokenizer, BertForSequenceClassification
#from transformers import AdamW
from torch.optim import AdamW
from transformers import get_linear_schedule_with_warmup
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
import pandas as pd
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

### data laoding

In [10]:
# Load dataset (replace path with your downloaded file)
df = pd.read_csv(r"C:\Users\Aoun Haider\Downloads\train.csv", names=['Class Index', 'Title', 'Description'], header=0)

print("data loading done")

data loading done


In [11]:
df.head()

,Class Index,Title,Description
0,3,Wall St. Bears Claw Back Into the Black (Reuters),"Reuters - Short-sellers, Wall Street's dwindli..."
1,3,Carlyle Looks Toward Commercial Aerospace (Reu...,Reuters - Private investment firm Carlyle Grou...
2,3,Oil and Economy Cloud Stocks' Outlook (Reuters),Reuters - Soaring crude prices plus worries\ab...
3,3,Iraq Halts Oil Exports from Main Southern Pipe...,Reuters - Authorities have halted oil export\f...
4,3,"Oil prices soar to all-time record, posing new...","AFP - Tearaway world oil prices, toppling reco..."


### preprocessing

In [12]:
# Preprocessing
# 1. Use only headlines (titles) for classification
# 2. Map labels: 1→0(World), 2→1(Sports), 3→2(Business), 4→3(Sci/Tech)
texts = df['Title'].tolist()
labels = [int(label)-1 for label in df['Class Index']]  # Convert 1-4 to 0-3

### test train split 

In [13]:
# Split data: 80% train, 10% validation, 10% test
train_texts, temp_texts, train_labels, temp_labels = train_test_split(
    texts, labels, test_size=0.2, random_state=42)
val_texts, test_texts, val_labels, test_labels = train_test_split(
    temp_texts, temp_labels, test_size=0.5, random_state=42)

### Tokenization 

In [14]:
# Initialize BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

# Tokenization function
def tokenize_data(text_list, max_length=64):
    input_ids = []
    attention_masks = []
    
    for text in text_list:
        encoded = tokenizer.encode_plus(
            text,
            add_special_tokens=True,  # Add [CLS] and [SEP]
            max_length=max_length,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'
        )
        input_ids.append(encoded['input_ids'])
        attention_masks.append(encoded['attention_mask'])
    
    return torch.cat(input_ids, dim=0), torch.cat(attention_masks, dim=0)

### tokenizing all dataset

In [15]:
# Tokenize all datasets
train_ids, train_masks = tokenize_data(train_texts)
val_ids, val_masks = tokenize_data(val_texts)
test_ids, test_masks = tokenize_data(test_texts)

### lables to tensors 

In [16]:
# Convert labels to tensors
train_labels = torch.tensor(train_labels)
val_labels = torch.tensor(val_labels)
test_labels = torch.tensor(test_labels)

In [17]:
# Create DataLoaders
batch_size = 32

train_data = TensorDataset(train_ids, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

val_data = TensorDataset(val_ids, val_masks, val_labels)
val_sampler = SequentialSampler(val_data)
val_dataloader = DataLoader(val_data, sampler=val_sampler, batch_size=batch_size)

# Initialize BERT model for classification
model = BertForSequenceClassification.from_pretrained(
    'bert-base-uncased',
    num_labels=4,  # 4 news categories
    output_attentions=False,
    output_hidden_states=False
)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### training

In [18]:
import torch

print("PyTorch version:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU Name:", torch.cuda.get_device_name(0))

PyTorch version: 2.7.1+cu128
CUDA available: True
GPU Name: NVIDIA GeForce RTX 4050 Laptop GPU


In [19]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [21]:
from tqdm import tqdm
import time
import torch

# Configure training parameters
optimizer = AdamW(model.parameters(), lr=2e-5, eps=1e-8)
epochs = 3
total_steps = len(train_dataloader) * epochs
scheduler = get_linear_schedule_with_warmup(
    optimizer, 
    num_warmup_steps=0,
    num_training_steps=total_steps
)

# Training loop
for epoch in range(epochs):
    print(f"\n======== Epoch {epoch+1} / {epochs} ========")
    start_time = time.time()
    
    model.train()
    total_loss = 0
    
    train_iterator = tqdm(train_dataloader, desc="Training", leave=False)
    
    for batch in train_iterator:
        batch = tuple(t.to(device) for t in batch)
        input_ids, attention_mask, labels = batch

        outputs = model(
            input_ids,
            attention_mask=attention_mask,
            labels=labels
        )
        loss = outputs.loss
        total_loss += loss.item()

        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()

        # Update tqdm progress bar with current loss
        train_iterator.set_postfix(loss=loss.item())

    avg_train_loss = total_loss / len(train_dataloader)
    epoch_time = time.time() - start_time

    # Validation phase
    model.eval()
    val_accuracy = []
    val_f1 = []

    val_iterator = tqdm(val_dataloader, desc="Validation", leave=False)

    for batch in val_iterator:
        batch = tuple(t.to(device) for t in batch)
        input_ids, attention_mask, labels = batch

        with torch.no_grad():
            outputs = model(input_ids, attention_mask=attention_mask)

        logits = outputs.logits
        preds = torch.argmax(logits, dim=1).flatten()

        preds = preds.cpu().numpy()
        labels = labels.cpu().numpy()

        val_accuracy.append(accuracy_score(labels, preds))
        val_f1.append(f1_score(labels, preds, average='macro'))

    print(f"Epoch {epoch+1}/{epochs}")
    print(f"Train Loss: {avg_train_loss:.4f}")
    print(f"Val Accuracy: {sum(val_accuracy)/len(val_accuracy):.4f}")
    print(f"Val F1 Score: {sum(val_f1)/len(val_f1):.4f}")
    print(f"Time Taken: {epoch_time:.2f} seconds")

    # Optional: Print GPU memory usage
    print(f"GPU Memory Allocated: {torch.cuda.memory_allocated(device)/1e6:.2f} MB")
    print(f"GPU Memory Cached:    {torch.cuda.memory_reserved(device)/1e6:.2f} MB")
    print("-" * 50)



======== Epoch 1 / 3 ========


Epoch 1/3
Train Loss: 0.2462
Val Accuracy: 0.9022
Val F1 Score: 0.8952
Time Taken: 782.60 seconds
GPU Memory Allocated: 1355.06 MB
GPU Memory Cached:    2998.93 MB
--------------------------------------------------

======== Epoch 2 / 3 ========


Epoch 2/3
Train Loss: 0.1649
Val Accuracy: 0.9047
Val F1 Score: 0.8972
Time Taken: 754.40 seconds
GPU Memory Allocated: 1355.06 MB
GPU Memory Cached:    2998.93 MB
--------------------------------------------------

======== Epoch 3 / 3 ========


Epoch 3/3
Train Loss: 0.1111
Val Accuracy: 0.9017
Val F1 Score: 0.8942
Time Taken: 741.69 seconds
GPU Memory Allocated: 1355.06 MB
GPU Memory Cached:    2998.93 MB
--------------------------------------------------


In [22]:
# Add after your training loop but before you close the notebook
from transformers import BertTokenizer, BertForSequenceClassification

# 1. Create save directory
import os
save_dir = "./bert_news_classifier"
os.makedirs(save_dir, exist_ok=True)

# 2. Save model and tokenizer
model.save_pretrained(save_dir)
tokenizer.save_pretrained(save_dir)

print(f"Model saved to {save_dir}")
print("Contents:", os.listdir(save_dir))

Model saved to ./bert_news_classifier
Contents: ['config.json', 'model.safetensors', 'special_tokens_map.json', 'tokenizer_config.json', 'vocab.txt']
